# Application 1

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pandas as pd
from iddn import iddn
from ddn3 import tools_export, visualize2

%load_ext autoreload
%autoreload 2

In [2]:
dat_folder = "../../../x_data/iddn/"

f0 = dat_folder + "GeneEntity.txt"
gene_names = pd.read_csv(f0, header=None)[0].to_numpy()

f0 = dat_folder + "GeneTF.txt"
tf_names = pd.read_csv(f0, header=None)[0].to_numpy()
tf_names = np.array(["TF_"+x for x in tf_names])

f0 = dat_folder + "dataHRDp.csv"
dat_pos = pd.read_csv(f0, header=None).to_numpy()
f0 = dat_folder + "dataHRDn.csv"
dat_neg = pd.read_csv(f0, header=None).to_numpy()

node_names = list(gene_names) + list(tf_names)

n_gene = len(gene_names)
n_tf = len(tf_names)

In [154]:
# dat_pos0 = dat_pos[:19, n_gene:]
# dat_neg0 = dat_neg[:19, n_gene:]
dat_pos0 = dat_pos[:, n_gene:]
dat_neg0 = dat_neg[:, n_gene:]

# n_sel = 100
# 
# n_pos = len(dat_pos0)
# idx_pos = np.random.choice(n_pos, n_sel, replace=True)
# # idx_pos = np.random.choice(n_pos, n_sel, replace=False)
# dat_pos0 = dat_pos0[idx_pos]
# 
# n_neg = len(dat_neg0)
# idx_neg = np.random.choice(n_neg, n_sel, replace=True)
# # idx_neg = np.random.choice(n_neg, n_sel, replace=False)
# dat_neg0 = dat_neg0[idx_neg]

n_node = n_gene + n_tf
if 0:  # same penalties for all edges
    scl = 1.2
    rho1_mat = np.ones((n_node, n_node))*0.5*scl
    rho1_mat[:n_gene,n_gene:] = 0.3*scl
    rho1_mat[n_gene:,:n_gene] = 0.3*scl
    rho1_mat[n_gene:, n_gene:] = 0.3*scl
else:
    rho1_mat = np.ones((n_node, n_node))*0.25

# rho2_mat = np.ones((n_node, n_node))*0.00
rho2_mat = np.ones((n_node, n_node))*0.05
dep_mat = np.ones((n_node, n_node))

# plt.imshow(rho1_mat)
# plt.clim([0,1])
# plt.colorbar()

In [155]:
g_rec = iddn.iddn(dat_pos0, dat_neg0, lambda1=rho1_mat, lambda2=rho2_mat, mthd='resi', dep_mat=dep_mat)

# np.sum(np.abs(g_rec)>1e-8)
comm_edge, _, _, diff_edge, node_show = tools_export.get_diff_comm_net_for_plot(g_rec[0], g_rec[1], node_names, thr=1e-4)
print(len(comm_edge), len(diff_edge))

nodes_type = dict()
for i in range(len(node_names)):
    if i < n_gene:
        nodes_type[node_names[i]] = 1
    else:
        nodes_type[node_names[i]] = 0


In [162]:
# g0 = g_rec[0]
# g1 = g_rec[1]
# 
# m0 = 1*(np.abs(g0)>1e-8)
# m1 = 1*(np.abs(g1)>1e-8)
# m01 = 1*((m0+m1)==2)
# 
# print(np.sum(m0), np.sum(m1), np.sum(m01))


In [167]:
np.sum(diff_edge['condition']==1)

In [161]:
visualize2.draw_network_for_ddn(
    comm_edge,
    node_show,
    mode="common",
    nodes_type=nodes_type,
    cen_lst=np.array([[-0.8,0],[0.8,0]]),
    rad_lst=np.array([[0.3,0.5],[0.8,0.8]]),
    labels=None,
    fig_size=(16, 9),
    font_size_scale=1,
    node_size_scale=1,
    min_alpha=0.4,
    dashed=False,
    pdf_name="./temp/",
)

In [160]:
visualize2.draw_network_for_ddn(
    diff_edge,
    node_show,
    mode="diff",
    nodes_type=nodes_type,
    cen_lst=np.array([[-0.8,0],[0.8,0]]),
    rad_lst=np.array([[0.3,0.5],[0.7,0.8]]),
    labels=None,
    fig_size=(16, 9),
    font_size_scale=1,
    node_size_scale=1,
    min_alpha=0.05,
    dashed=False,
    pdf_name="./temp/",
)